In [133]:
import pandas as pd

import os
from pathlib import Path
import re


In [134]:
root_folder = Path("./..").resolve()
datasets_folder = root_folder / 'cleaned_data'

In [135]:
speeches_file = datasets_folder / "all_speeches_2013_2022.parquet"
corruption_file = datasets_folder / "corruption_index.parquet"
fos_file = datasets_folder / "freedom_of_the_press_2013_2023.parquet"

In [136]:
speeches = pd.read_parquet(speeches_file)
corruption = pd.read_parquet(corruption_file)
fos = pd.read_parquet(fos_file)

In [137]:
combined = pd.merge(left=corruption, right=fos, on=["country_name", "iso_3", "year"], how="right")

In [138]:
combined = combined.merge(speeches, on=["iso_3", "year"], how="left")

In [160]:
def replace_new_paragraph_line(text):
    pattern = re.compile("[\. ! \?]\n[A-Z]")
    matches = re.finditer(pattern, text)
    positions = [match.start() for match in matches]
    initial_position = 0
    text_list = []
    for end_position in positions[1:]:
        text_list.append(text[initial_position:end_position])
        initial_position = end_position + 2
    text_list.append(text[initial_position:])
    return text_list

In [145]:
combined_before_2023 = combined.loc[combined['year'] < 2023]

Countries that are missing from UN speeches during the years

In [149]:
combined_before_2023[combined_before_2023['text'].isna()]

,country_name,iso_3,Region,corruption_sources,year,corruption_scores,corruption_standard_errors,global_score,fos_yearly_evolution,text
22,Afghanistan,AFG,AP,5.0,2021,16.0,5.0,59.81,-2.49,NaN
192,Taiwan,TWN,AP,8.0,2022,68.0,8.0,74.00,-2.14,NaN
279,OECS,XCD,NaN,NaN,2022,NaN,NaN,68.00,0.00,NaN
309,Kosovo,XKO,NaN,NaN,2022,NaN,NaN,67.00,0.00,NaN
409,Cyprus North,CTU,NaN,NaN,2022,NaN,NaN,61.00,0.00,NaN
438,Brunei Darussalam,BRN,AP,3.0,2016,58.0,3.0,46.15,-17.09,NaN
448,Brunei Darussalam,BRN,AP,3.0,2020,60.0,3.0,50.35,1.83,NaN
482,Myanmar,MMR,AP,7.0,2021,28.0,7.0,53.86,-1.37,NaN
527,Cameroon,CMR,SSA,8.0,2015,27.0,8.0,60.37,-1.50,NaN
631,Hong Kong,HKG,AP,8.0,2022,76.0,8.0,41.00,0.00,NaN


In [152]:
combined_before_2023_given_speech = combined_before_2023[combined_before_2023['text'].notna()]

In [163]:
combined_before_2023_given_speech.loc[:,:]["text_split"] = combined_before_2023_given_speech['text'].apply(lambda x: replace_new_paragraph_line(x)).apply(lambda x: x[1:])

In [166]:
combined_before_2023_given_speech = combined_before_2023_given_speech.drop("text", axis=1)

In [174]:
combined_before_2023_given_speech.explode("text_split").to_parquet("combined_data_2013_2022_per_paragraph.parquet")

In [175]:
combined.to_parquet("combined_data_2013_2023.parquet")

In [176]:
combined_before_2023.to_parquet("combined_data_2013_2022_with_missing_speeches.parquet")